In [ ]:
# @title
from IPython.display import display, HTML

display(HTML("""
<script>
const firstCell = document.querySelector('.cell.code_cell');
if (firstCell) {
  firstCell.querySelector('.input').style.pointerEvents = 'none';
  firstCell.querySelector('.input').style.opacity = '0.5';
}
</script>
"""))

html = """
<div style="display:flex; flex-direction:column; align-items:center; text-align:center; gap:12px; padding:8px;">
  <h1 style="margin:0;">👋 Welcome to <span style="color:#1E88E5;">Algopath Coding Academy</span>!</h1>

  <img src="https://raw.githubusercontent.com/sshariqali/mnist_pretrained_model/main/algopath_logo.jpg"
       alt="Algopath Coding Academy Logo"
       width="400"
       style="border-radius:15px; box-shadow:0 4px 12px rgba(0,0,0,0.2); max-width:100%; height:auto;" />

  <p style="font-size:16px; margin:0;">
    <em>Empowering young minds to think creatively, code intelligently, and build the future with AI.</em>
  </p>
</div>
"""

display(HTML(html))

## Day 8 - Part 6: Add & Norm (Residual Connections & Layer Normalization)

---

### **Continuing from Part 5**

We built a complete Transformer Block with:
- ✅ Multi-Head Attention (communication)
- ✅ Feed-Forward Network (computation)

**The Problem:** Deep networks are hard to train!
- Gradients can vanish (become too small)
- Gradients can explode (become too large)
- Training becomes unstable

---

### **Agenda for this Notebook**

| Section | Topic | Description |
|:-------:|-------|-------------|
| 1 | **Setup** | Same data pipeline |
| 2 | **Existing Components** | Head, MultiHead, FFN |
| 3 | **🆕 Layer Normalization** | Stabilize activations |
| 4 | **🆕 Residual Connections** | Help gradients flow |
| 5 | **Updated Block** | Add & Norm in the block |
| 6 | **Training & Generation** | Stable training! |

---

### **Two Critical Techniques**

**1. Residual Connections (Skip Connections)**

Instead of: `output = f(x)`
We do: `output = x + f(x)`

```
     ┌─────────────────┐
     │                 │
x ───┼──→ Attention ───┼──→ x + Attention(x)
     │                 │
     └────────+────────┘
              ↑
        Skip Connection
```

**Why?** Gradients can flow directly through the skip connection, avoiding vanishing gradients!

**2. Layer Normalization**

Normalizes activations to have mean=0, variance=1:

$$\text{LayerNorm}(x) = \gamma \cdot \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$$

**Why?** Keeps activations in a reasonable range, preventing exploding values!

Let's implement these critical components! 🚀

---
## Section 1: Setup (Same as Before)

In [ ]:
import torch

Hyperparameters:

In [2]:
batch_size = 32 # how many independent sequences to process in parallel
block_size = 8 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32

---
## Section 2: Data Pipeline

In [3]:
# Load the tiny shakespeare dataset
dataset = "tiny_shakespeare.txt"

# Load the dataset into a string
with open(dataset, "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

Tokenizer:

In [4]:
stoi = { ch: i for i, ch in enumerate(chars)}
itos = { i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

Data split:

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

Batch loading:

In [6]:
torch.manual_seed(1337)

def get_batch(split):
    # generate a small batch of input-target pairs
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

Loss estimation:

In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

---
## Section 3: Attention Components (Review)

In [8]:
class Head(torch.nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = torch.nn.Linear(n_embed, head_size, bias=False)
        self.query = torch.nn.Linear(n_embed, head_size, bias=False)
        self.value = torch.nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = torch.nn.functional.softmax(wei, dim=-1)
        out = wei @ v
        return out

Multi-Head Attention:

In [9]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = torch.nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = torch.nn.Linear(n_embed, n_embed)
    
    def forward(self, x):
        return self.proj(torch.cat([h(x) for h in self.heads], dim=-1))

Feed-Forward Network:

In [10]:
class FeedForward(torch.nn.Module):

    def __init__(self, n_embed):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(n_embed, n_embed * 4),
            torch.nn.ReLU(),
            torch.nn.Linear(n_embed * 4, n_embed)
        )
    
    def forward(self, x):
        return self.net(x)

---
## 🆕 Section 4: Layer Normalization

**Understanding Layer Norm:**

Layer Normalization normalizes across the **feature dimension** for each token independently.

**Formula:**
$$\hat{x} = \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}}$$
$$\text{output} = \gamma \cdot \hat{x} + \beta$$

**Where:**
- $\mu$: Mean across features
- $\sigma$: Standard deviation across features
- $\gamma$, $\beta$: Learnable scale and shift parameters

**Why LayerNorm (not BatchNorm)?**
- BatchNorm normalizes across the batch → requires large batches
- LayerNorm normalizes across features → works with any batch size
- Better for sequence models where batch size varies

Here's a simple implementation to understand the concept (we'll use PyTorch's built-in version in practice):

In [11]:
class LayerNorm:

    def __init__(self, dim, eps = 1e-5, momentum = 0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        x_mean = x.mean(1, keepdim=True)
        x_var = x.var(1, keepdim=True)
        x_hat = (x - x_mean) / torch.sqrt(x_var + self.eps)
        return self.gamma * x_hat + self.beta

---
## 🆕 Section 5: Updated Block with Add & Norm

**The key changes to our Block:**

```python
# Before (Part 5):
def forward(self, x):
    x = self.sa(x)    # Just attention
    x = self.ffwd(x)  # Just FFN
    return x

# Now (Part 6):
def forward(self, x):
    x = x + self.sa(self.ln1(x))    # LayerNorm → Attention → Residual
    x = x + self.ffwd(self.ln2(x))  # LayerNorm → FFN → Residual
    return x
```

**Note:** We use "Pre-Norm" architecture (norm BEFORE attention/FFN).
- Original Transformer used "Post-Norm" (norm after)
- Pre-Norm trains more stably and is now standard!

**Visual Comparison:**
```
Part 5:                      Part 6 (Pre-Norm):
x → Attention → FFN → out    x → LN → Attention → + → LN → FFN → + → out
                                 └────────────────┘   └──────────┘
                                     Residual            Residual
```

In [12]:
class Block(torch.nn.Module):

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = torch.nn.LayerNorm(n_embed)
        self.ln2 = torch.nn.LayerNorm(n_embed)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # residual connection
        x = x + self.ffwd(self.ln2(x)) # residual connection
        return x

---
## Section 6: Transformer Model

The model remains similar, just using our improved Block:

In [ ]:
torch.manual_seed(1337)

class Transformer(torch.nn.Module):

    def __init__(self):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = torch.nn.Embedding(block_size, n_embed)
        self.block = Block(n_embed, n_head = 4)
        self.lm_head = torch.nn.Linear(n_embed, vocab_size)

    def forward(self, idx, y = None):
        # idx is (B, T) tensor of indices.
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, n_embed)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device))
        x = tok_emb + pos_emb
        x = self.block(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        # Loss
        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = torch.functional.F.cross_entropy(logits, y)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) tensor of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = torch.nn.functional.softmax(logits, dim = -1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx

Initialize and move to device:

In [ ]:
model = Transformer()
model = model.to(device)

---
## Section 7: Training

With residual connections and layer norm, training should be more stable:

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [16]:
for i in range(max_iters):
    
    if i % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)

    # perform backpropagation
    loss.backward()

    # update the weights
    optimizer.step()

    # zero the gradients
    optimizer.zero_grad(set_to_none = True)

step 0: train loss 4.4829, val loss 4.4847
step 300: train loss 2.5911, val loss 2.6113
step 600: train loss 2.4491, val loss 2.4611
step 900: train loss 2.3795, val loss 2.3911
step 1200: train loss 2.3252, val loss 2.3444
step 1500: train loss 2.2872, val loss 2.3080
step 1800: train loss 2.2643, val loss 2.2824
step 2100: train loss 2.2370, val loss 2.2516
step 2400: train loss 2.2135, val loss 2.2453
step 2700: train loss 2.2069, val loss 2.2390
step 3000: train loss 2.1922, val loss 2.2167
step 3300: train loss 2.1854, val loss 2.2059
step 3600: train loss 2.1651, val loss 2.1953
step 3900: train loss 2.1544, val loss 2.2013
step 4200: train loss 2.1530, val loss 2.2037
step 4500: train loss 2.1539, val loss 2.1900
step 4800: train loss 2.1206, val loss 2.1901
step 5100: train loss 2.1241, val loss 2.1641
step 5400: train loss 2.0929, val loss 2.1721
step 5700: train loss 2.1035, val loss 2.1505
step 6000: train loss 2.0970, val loss 2.1531
step 6300: train loss 2.0912, val loss 2

---
## Section 8: Text Generation

In [18]:
context = torch.zeros((1, 1), dtype = torch.long) # Since idx 0 is a new line character
out = model.generate(context, max_new_tokens = 200)
print(decode(out[0].tolist()))


And in slaw.

MEO:
Nak oukess, and my werigh the weemble:
Do lowTle and sus but with ange in thun and by I kinge.
Then:
Wey, the upust he baring:
Orred,
An poof I shom thou dresself, good, garef; ourp


---
## 📝 Summary & What's Next

### What We Added:
- ✅ **Layer Normalization**: Stabilizes activations across features
- ✅ **Residual Connections**: Enables gradient flow in deep networks
- ✅ **Pre-Norm Architecture**: Modern, stable design pattern

### Current Architecture:
```
Token Embed + Pos Embed → [LN → MHA → + → LN → FFN → +] → LM Head → Output
                         └──────── Block with Add & Norm ────────┘
```

### The Final Step: SCALING UP!

We've built all the components of a real Transformer:
- ✅ Token & Position Embeddings
- ✅ Multi-Head Attention
- ✅ Feed-Forward Network
- ✅ Layer Normalization
- ✅ Residual Connections

**But our model is tiny:**
- `n_embed = 32` (GPT-3: 12288)
- `block_size = 8` (GPT-3: 2048)
- 1 block (GPT-3: 96 blocks)

---

### ➡️ Next: Part 7 - Scaling Up to a Real GPT!

We'll scale up ALL the parameters:
- More embedding dimensions
- Longer context
- Multiple stacked blocks
- Dropout for regularization

Let's build a real language model! 🚀